Have to:


*   Import
*   Remove unimportant columns
*   Remove rows that have NaN in 72 (pol affil)
*   Then we score everything besides the likert scale questions
*   Last we change the likert scale NaN's to zeroes, and score them
*   Store all scores seperate then add to seperate database



Importing data

In [ ]:
##Evan Lysko
##4/12/2021
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


data = pd.read_csv("/content/drive/MyDrive/Internship/Python/All_Data.csv")
data

,Respondent ID,Collector ID,Start Date,End Date,IP Address,Email Address,First Name,Last Name,Custom Data 1,"Upon checking the box below, I certify that I am 18 years of age or older and willingly consent to participate in this study.",I sometimes find it difficult to see things from the ‘other guy’s’ point of view.,I try to look at everybody’s side of a disagreement before I make a decision.,I sometimes try to understand my friends better by imagining how things look from their perspective.,"When I am upset at someone, I usually try to ‘put myself in his shoes' for a while.","Before criticizing somebody, I try to imagine how I would feel if I was in their place.",I often get emotionally involved with my friends’ problems.,I am inclined to get nervous when others around me seem to be nervous.,People I am with have a strong influence on my mood.,It affects me very much when one of my friends seems upset.,"I often get deeply involved with the feelings of a character in a film, play or novel.",I get very upset when I see someone cry.,I am happy when I am with a cheerful group and sad when the others are glum.,It worries me when others are worrying and panicky.,I can easily tell if someone else wants to enter a conversation.,I can pick up quickly if someone says one thing but means another.,It is hard for me to see why some things upset people so much.,I find it easy to put myself in somebody else's shoes.,I am good at predicting how someone will feel.,I am quick to spot when someone in a group is feeling awkward or uncomfortable.,Other people tell me I am good at understanding how they are feeling and what they are thinking.,I can easily tell if someone else is interested or bored with what I am saying.,Friends talk to me about their problems as they say that I am very understanding.,"I can sense if I am intruding, even if the other person does not tell me.",I can easily work out what another person might want to talk about.,I can tell if someone is masking their true emotion.,I am good at predicting what someone will do.,"I can usually appreciate the other person's viewpoint, even if I do not agree with it.",I usually stay emotionally detached when watching a film.,I always try to consider other's feelings before I do something.,Before I do something I try to consider how my friends will react to it.,...,I am someone who has a forgiving nature.,I am someone who is helpful and unselfish with others.,I am someone who is sometimes rude to others.,I am someone who is suspicious of others’ intentions.,I am someone who can be cold and uncaring.,"I am someone who is polite, courteous to others.",I am someone who assumes the best about people.,People see me as a natural leader.,I hate being the center of attention.,Many group activities tend to be dull without me.,I know that I am special because everyone keeps telling me so.,I like to get acquainted with important people.,I feel embarrassed if someone compliments me.,I have been compared to famous people.,I am an average person.,I insist on getting the respect I deserve.,I like to get revenge on authorities.,I avoid dangerous situations.,Payback needs to be quick and nasty.,People often say I’m out of control.,It’s true that I can be mean to others.,People who mess with me always regret it.,I have never gotten into trouble with the law.,I’ll say anything to get what I want.,"In general, how likely are you to follow the rules? (0 = no never, 100 = yes, always)","Is it ok for teenagers to drink alcohol? (0 = no never, 100 = yes, always)","If there is a mask mandate in response to COVID-19 how likely are you to wear a mask in public?(0 = no never, 100 = yes, always)","How likely are you to drive over the speed limit? (0 = no never, 100 = yes, always)","Is it ever ok for people to steal from a store? (0 = no never, 100 = yes, always)","How likely are you to lie to someone if it benefits you? (0 = no never, 100 = yes, always)","How likely are you to cut in line somewhere? (0 = no never, 100

Cleaning up the data by removing the unecessary columns and rows.

In [ ]:
drop_columns = ["Collector ID", "Start Date", "End Date", "IP Address", "Email Address", "First Name",	"Last Name",	"Custom Data 1", "Unnamed: 84"]
clean_data = data.drop(drop_columns, axis=1)
clean_data = clean_data.drop([0])
clean_data
clean_data.index


Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            216, 217, 218, 219, 220, 221, 222, 223, 224, 225],
           dtype='int64', length=225)

Now we need to figure out what do to with unanswered questions (NaN). Can either get rid of any that skipped, make skip=0 so it will be neutral, or set a limit for how many skips there can be before a response is thrown out.

In [ ]:
clean_data.iloc[:, 72]

1      NaN
2        1
3        1
4      NaN
5      NaN
      ... 
221      1
222      4
223      2
224      2
225    NaN
Name: What is your political affiliation?, Length: 225, dtype: object

Make this if it == NaN, not zero, that way I can get rid of useless rows without effecting slider scores by changing to zero.

In [ ]:
clean_data = clean_data[clean_data['What is your political affiliation?'].notna()]
clean_data

,Respondent ID,"Upon checking the box below, I certify that I am 18 years of age or older and willingly consent to participate in this study.",I sometimes find it difficult to see things from the ‘other guy’s’ point of view.,I try to look at everybody’s side of a disagreement before I make a decision.,I sometimes try to understand my friends better by imagining how things look from their perspective.,"When I am upset at someone, I usually try to ‘put myself in his shoes' for a while.","Before criticizing somebody, I try to imagine how I would feel if I was in their place.",I often get emotionally involved with my friends’ problems.,I am inclined to get nervous when others around me seem to be nervous.,People I am with have a strong influence on my mood.,It affects me very much when one of my friends seems upset.,"I often get deeply involved with the feelings of a character in a film, play or novel.",I get very upset when I see someone cry.,I am happy when I am with a cheerful group and sad when the others are glum.,It worries me when others are worrying and panicky.,I can easily tell if someone else wants to enter a conversation.,I can pick up quickly if someone says one thing but means another.,It is hard for me to see why some things upset people so much.,I find it easy to put myself in somebody else's shoes.,I am good at predicting how someone will feel.,I am quick to spot when someone in a group is feeling awkward or uncomfortable.,Other people tell me I am good at understanding how they are feeling and what they are thinking.,I can easily tell if someone else is interested or bored with what I am saying.,Friends talk to me about their problems as they say that I am very understanding.,"I can sense if I am intruding, even if the other person does not tell me.",I can easily work out what another person might want to talk about.,I can tell if someone is masking their true emotion.,I am good at predicting what someone will do.,"I can usually appreciate the other person's viewpoint, even if I do not agree with it.",I usually stay emotionally detached when watching a film.,I always try to consider other's feelings before I do something.,Before I do something I try to consider how my friends will react to it.,"I am someone who is compassionate, has a soft heart.","I am someone who is respectful, treats others with respect.",I am someone who tends to find fault with others.,I am someone who feels little sympathy for others.,I am someone who starts arguments with others.,I am someone who has a forgiving nature.,I am someone who is helpful and unselfish with others.,I am someone who is sometimes rude to others.,I am someone who is suspicious of others’ intentions.,I am someone who can be cold and uncaring.,"I am someone who is polite, courteous to others.",I am someone who assumes the best about people.,People see me as a natural leader.,I hate being the center of attention.,Many group activities tend to be dull without me.,I know that I am special because everyone keeps telling me so.,I like to get acquainted with important people.,I feel embarrassed if someone compliments me.,I have been compared to famous people.,I am an average person.,I insist on getting the respect I deserve.,I like to get revenge on authorities.,I avoid dangerous situations.,Payback needs to be quick and nasty.,People often say I’m out of control.,It’s true that I can be mean to others.,People who mess with me always regret it.,I have never gotten into trouble with the law.,I’ll say anything to get what I want.,"In general, how likely are you to follow the rules? (0 = no never, 100 = yes, always)","Is it ok for teenagers to drink alcohol? (0 = no never, 100 = yes, always)","If there is a mask mandate in response to COVID-19 how likely are you to wear a mask in public?(0 = no never, 100 = yes, always)","How likely are you to drive over the speed limit? (0 = no never, 100 = yes, always)","Is it ever ok for people to steal from a store? (0 = no never, 100 = yes

Slicing into empathy, aggreeableness, and so on.

In [ ]:
scores = pd.DataFrame(clean_data['What is your political affiliation?'])
scores['Vote'] = clean_data.iloc[:, 73]
scores['Age'] = clean_data.iloc[:, 74]
scores['Gender'] = clean_data.iloc[:, 75]
scores['Stay at home'] = clean_data.iloc[:, 70]
scores['Social distancing'] = clean_data.iloc[:, 68]
scores['Mask wearing'] = clean_data.iloc[:, 63]
scores

,What is your political affiliation?,Vote,Age,Gender,Stay at home,Social distancing,Mask wearing
2,1,1,4,1,5,98,100
3,1,1,2,1,NaN,75,25
6,1,1,3,1,80,25,5
7,2,2,4,1,90,97,97
8,1,1,3,1,34,58,100
...,...,...,...,...,...,...,...
218,1,1,2,1,75,75,100
221,1,1,5,1,50,100,100
222,4,6,1,2,90,100,100
223,2,2,2,1,95,100,100


In [ ]:
clean_likert = clean_data.fillna(0)
clean_likert

,Respondent ID,"Upon checking the box below, I certify that I am 18 years of age or older and willingly consent to participate in this study.",I sometimes find it difficult to see things from the ‘other guy’s’ point of view.,I try to look at everybody’s side of a disagreement before I make a decision.,I sometimes try to understand my friends better by imagining how things look from their perspective.,"When I am upset at someone, I usually try to ‘put myself in his shoes' for a while.","Before criticizing somebody, I try to imagine how I would feel if I was in their place.",I often get emotionally involved with my friends’ problems.,I am inclined to get nervous when others around me seem to be nervous.,People I am with have a strong influence on my mood.,It affects me very much when one of my friends seems upset.,"I often get deeply involved with the feelings of a character in a film, play or novel.",I get very upset when I see someone cry.,I am happy when I am with a cheerful group and sad when the others are glum.,It worries me when others are worrying and panicky.,I can easily tell if someone else wants to enter a conversation.,I can pick up quickly if someone says one thing but means another.,It is hard for me to see why some things upset people so much.,I find it easy to put myself in somebody else's shoes.,I am good at predicting how someone will feel.,I am quick to spot when someone in a group is feeling awkward or uncomfortable.,Other people tell me I am good at understanding how they are feeling and what they are thinking.,I can easily tell if someone else is interested or bored with what I am saying.,Friends talk to me about their problems as they say that I am very understanding.,"I can sense if I am intruding, even if the other person does not tell me.",I can easily work out what another person might want to talk about.,I can tell if someone is masking their true emotion.,I am good at predicting what someone will do.,"I can usually appreciate the other person's viewpoint, even if I do not agree with it.",I usually stay emotionally detached when watching a film.,I always try to consider other's feelings before I do something.,Before I do something I try to consider how my friends will react to it.,"I am someone who is compassionate, has a soft heart.","I am someone who is respectful, treats others with respect.",I am someone who tends to find fault with others.,I am someone who feels little sympathy for others.,I am someone who starts arguments with others.,I am someone who has a forgiving nature.,I am someone who is helpful and unselfish with others.,I am someone who is sometimes rude to others.,I am someone who is suspicious of others’ intentions.,I am someone who can be cold and uncaring.,"I am someone who is polite, courteous to others.",I am someone who assumes the best about people.,People see me as a natural leader.,I hate being the center of attention.,Many group activities tend to be dull without me.,I know that I am special because everyone keeps telling me so.,I like to get acquainted with important people.,I feel embarrassed if someone compliments me.,I have been compared to famous people.,I am an average person.,I insist on getting the respect I deserve.,I like to get revenge on authorities.,I avoid dangerous situations.,Payback needs to be quick and nasty.,People often say I’m out of control.,It’s true that I can be mean to others.,People who mess with me always regret it.,I have never gotten into trouble with the law.,I’ll say anything to get what I want.,"In general, how likely are you to follow the rules? (0 = no never, 100 = yes, always)","Is it ok for teenagers to drink alcohol? (0 = no never, 100 = yes, always)","If there is a mask mandate in response to COVID-19 how likely are you to wear a mask in public?(0 = no never, 100 = yes, always)","How likely are you to drive over the speed limit? (0 = no never, 100 = yes, always)","Is it ever ok for people to steal from a store? (0 = no never, 100 = yes

In [ ]:
empathy_items = pd.DataFrame(clean_likert.iloc[:,2:32])
empathy_items = empathy_items.apply(pd.to_numeric, errors='coerce')
empathy_items

,I sometimes find it difficult to see things from the ‘other guy’s’ point of view.,I try to look at everybody’s side of a disagreement before I make a decision.,I sometimes try to understand my friends better by imagining how things look from their perspective.,"When I am upset at someone, I usually try to ‘put myself in his shoes' for a while.","Before criticizing somebody, I try to imagine how I would feel if I was in their place.",I often get emotionally involved with my friends’ problems.,I am inclined to get nervous when others around me seem to be nervous.,People I am with have a strong influence on my mood.,It affects me very much when one of my friends seems upset.,"I often get deeply involved with the feelings of a character in a film, play or novel.",I get very upset when I see someone cry.,I am happy when I am with a cheerful group and sad when the others are glum.,It worries me when others are worrying and panicky.,I can easily tell if someone else wants to enter a conversation.,I can pick up quickly if someone says one thing but means another.,It is hard for me to see why some things upset people so much.,I find it easy to put myself in somebody else's shoes.,I am good at predicting how someone will feel.,I am quick to spot when someone in a group is feeling awkward or uncomfortable.,Other people tell me I am good at understanding how they are feeling and what they are thinking.,I can easily tell if someone else is interested or bored with what I am saying.,Friends talk to me about their problems as they say that I am very understanding.,"I can sense if I am intruding, even if the other person does not tell me.",I can easily work out what another person might want to talk about.,I can tell if someone is masking their true emotion.,I am good at predicting what someone will do.,"I can usually appreciate the other person's viewpoint, even if I do not agree with it.",I usually stay emotionally detached when watching a film.,I always try to consider other's feelings before I do something.,Before I do something I try to consider how my friends will react to it.
2,0,1,1,1,0,1,-1,0,1,-1,0,1,1,1,1,1,1,1,1,0,1,1,1,0,0,1,2,0,1,1
3,-1,1,1,1,1,1,-2,-1,1,-2,1,0,0,1,1,-1,1,1,1,1,1,2,1,1,1,1,2,1,2,1
6,2,-1,-1,-1,1,2,2,2,2,0,1,2,0,0,2,2,-1,-1,2,1,2,1,2,-2,-1,2,-1,0,-1,-1
7,0,1,2,1,2,1,1,1,1,0,1,1,1,2,-1,-1,1,-1,0,-1,1,2,1,-1,1,0,1,-1,2,1
8,-1,2,1,0,-1,1,-1,1,-1,1,1,2,-1,1,0,-1,0,0,1,1,1,1,1,0,1,-1,1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,-2,2,1,1,1,0,-1,1,1,0,1,1,0,2,1,1,1,1,1,1,2,1,1,2,2,-1,2,1,1,1
221,-2,2,1,1,2,1,1,2,2,-1,2,1,0,1,1,-1,1,1,1,0,1,1,1,0,1,0,1,-1,1,1
222,-2,1,2,1,0,-1,-2,1,0,-1,-1,1,-1,2,-1,-1,1,-1,2,2,1,1,1,-1,0,1,1,-1,1,0
223,-1,2,2,2,1,2,2,2,1,2,-1,2,1,1,2,1,1,2,1,1,-1,2,1,1,1,1,1,-2,1,1


In [ ]:
scores['Empathy'] = empathy_items.sum(axis=1)
scores

,What is your political affiliation?,Vote,Age,Gender,Stay at home,Social distancing,Mask wearing,Empathy
2,1,1,4,1,5,98,100,19
3,1,1,2,1,NaN,75,25,19
6,1,1,3,1,80,25,5,17
7,2,2,4,1,90,97,97,19
8,1,1,3,1,34,58,100,7
...,...,...,...,...,...,...,...,...
218,1,1,2,1,75,75,100,26
221,1,1,5,1,50,100,100,22
222,4,6,1,2,90,100,100,6
223,2,2,2,1,95,100,100,32


In [ ]:
agreeableness_items = pd.DataFrame(clean_likert.iloc[:,32:44])
agreeableness_items = agreeableness_items.apply(pd.to_numeric, errors='coerce')
agreeableness_items

,"I am someone who is compassionate, has a soft heart.","I am someone who is respectful, treats others with respect.",I am someone who tends to find fault with others.,I am someone who feels little sympathy for others.,I am someone who starts arguments with others.,I am someone who has a forgiving nature.,I am someone who is helpful and unselfish with others.,I am someone who is sometimes rude to others.,I am someone who is suspicious of others’ intentions.,I am someone who can be cold and uncaring.,"I am someone who is polite, courteous to others.",I am someone who assumes the best about people.
2,1,2,1,1,2,2,2,1,1,1,2,1
3,1,2,2,1,1,2,2,1,-1,2,2,1
6,2,1,-2,1,-1,-2,1,-2,-2,-1,1,-1
7,2,2,0,2,2,2,2,2,2,2,2,2
8,2,2,-1,-2,1,2,2,-1,-1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
218,1,2,1,2,1,-1,1,1,-1,1,2,0
221,2,2,2,2,2,2,1,2,2,2,2,2
222,1,2,0,2,-1,2,1,-1,-1,-1,1,0
223,1,2,-1,1,2,1,1,1,-1,0,1,-1


In [ ]:
scores['Agreeableness'] = agreeableness_items.sum(axis=1)
scores

,What is your political affiliation?,Vote,Age,Gender,Stay at home,Social distancing,Mask wearing,Empathy,Agreeableness
2,1,1,4,1,5,98,100,19,17
3,1,1,2,1,NaN,75,25,19,16
6,1,1,3,1,80,25,5,17,-5
7,2,2,4,1,90,97,97,19,22
8,1,1,3,1,34,58,100,7,6
...,...,...,...,...,...,...,...,...,...
218,1,1,2,1,75,75,100,26,10
221,1,1,5,1,50,100,100,22,23
222,4,6,1,2,90,100,100,6,5
223,2,2,2,1,95,100,100,32,7


In [ ]:
narcissism_items = pd.DataFrame(clean_likert.iloc[:,44:53])
narcissism_items = narcissism_items.apply(pd.to_numeric, errors='coerce')
narcissism_items

,People see me as a natural leader.,I hate being the center of attention.,Many group activities tend to be dull without me.,I know that I am special because everyone keeps telling me so.,I like to get acquainted with important people.,I feel embarrassed if someone compliments me.,I have been compared to famous people.,I am an average person.,I insist on getting the respect I deserve.
2,1,-2,-2,0,0,-1,0,-1,-2
3,2,1,0,0,0,1,0,2,1
6,2,1,1,2,0,-1,1,2,2
7,1,0,-2,-1,1,-1,-2,-2,-1
8,0,-2,-1,-1,-1,-1,-1,-1,1
...,...,...,...,...,...,...,...,...,...
218,1,-1,-1,0,0,1,0,0,0
221,1,0,0,0,0,-1,-1,1,1
222,2,1,1,2,2,1,2,1,-2
223,2,-2,-2,-2,-1,-2,-2,-2,1


In [ ]:
scores['Narcissism'] = narcissism_items.sum(axis=1)
scores

,What is your political affiliation?,Vote,Age,Gender,Stay at home,Social distancing,Mask wearing,Empathy,Agreeableness,Narcissism
2,1,1,4,1,5,98,100,19,17,-7
3,1,1,2,1,NaN,75,25,19,16,7
6,1,1,3,1,80,25,5,17,-5,10
7,2,2,4,1,90,97,97,19,22,-7
8,1,1,3,1,34,58,100,7,6,-7
...,...,...,...,...,...,...,...,...,...,...
218,1,1,2,1,75,75,100,26,10,0
221,1,1,5,1,50,100,100,22,23,1
222,4,6,1,2,90,100,100,6,5,10
223,2,2,2,1,95,100,100,32,7,-10


In [ ]:
psychopathy_items = pd.DataFrame(clean_likert.iloc[:,53:61])
psychopathy_items = psychopathy_items.apply(pd.to_numeric, errors='coerce')
psychopathy_items

,I like to get revenge on authorities.,I avoid dangerous situations.,Payback needs to be quick and nasty.,People often say I’m out of control.,It’s true that I can be mean to others.,People who mess with me always regret it.,I have never gotten into trouble with the law.,I’ll say anything to get what I want.
2,-2,-2,-2,-2,-2,-2,-2,-2
3,-1,-2,-2,-2,-2,-2,-1,-1
6,2,2,1,2,1,2,-2,1
7,-2,-2,-2,-2,-2,-2,-2,-2
8,-2,-2,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...
218,-1,-1,-2,-2,-2,-2,-2,-2
221,-2,-2,-2,-2,-2,-2,-2,-2
222,-2,-1,-2,0,1,-2,0,-1
223,1,-1,-1,-2,1,-2,-2,-1


In [ ]:
scores['Psychopathy'] = psychopathy_items.sum(axis=1)
scores

,What is your political affiliation?,Vote,Age,Gender,Stay at home,Social distancing,Mask wearing,Empathy,Agreeableness,Narcissism,Psychopathy
2,1,1,4,1,5,98,100,19,17,-7,-16
3,1,1,2,1,NaN,75,25,19,16,7,-13
6,1,1,3,1,80,25,5,17,-5,10,9
7,2,2,4,1,90,97,97,19,22,-7,-16
8,1,1,3,1,34,58,100,7,6,-7,-10
...,...,...,...,...,...,...,...,...,...,...,...
218,1,1,2,1,75,75,100,26,10,0,-14
221,1,1,5,1,50,100,100,22,23,1,-16
222,4,6,1,2,90,100,100,6,5,10,-7
223,2,2,2,1,95,100,100,32,7,-10,-7


In [ ]:
scores.to_csv("/content/drive/MyDrive/Internship/Python/scores.csv")